In [1]:
from main import *
import numpy as np
import pandas as pd

# parameter setup

In [2]:
target_feature = "merged_support3_random_1k_seed_42"
target_feature_suffix = "_matrix.npy"
save_data_path = "./results"

select_methods = ["rf"]# ["random", "xgb", "rf", "variance", "chi2", "f_classif"] # Extra-trees # "mutual_info_classif"
    
n_select_list = [256, 512, 1024, 2048, 4096, 8192, 16384, 32768, 65536, 131072] #5105448
n_dim_reduce_list = [128, 256, 512, 1024, None]  ## list should always contain None to perform whole feature training after selection
ML_models = ["SVM"] #["SVM", "XGB", "RF", "DT", "KNN"]

# data loading

In [3]:
feature_data_path, sample_annotation_file = get_data_path(save_data_path)

In [4]:
dataset = data_loader(os.path.join(feature_data_path, target_feature + target_feature_suffix), 
                        sample_annotation_file)
(X, y_original, y), (train_indices, val_indices, test_indices), label_mapping = dataset.get_data()


result_combined = []

2024-06-24 11:02:42,178 - INFO - [progress] Read data done. X.shape: (3202, 1000), y.shape: (3202,)
2024-06-24 11:02:42,181 - INFO - [progress] Dropped 1 samples from the dataset. X.shape: (3201, 1000), y.shape: (3201,)


2024-06-24 11:02:42,185 - INFO -  - Data_split: train_set (n= 1920), val_set (n= 640), test_set (n= 641)


# save feature selection result

In [ ]:
n = 8192
save_file_name = os.path.join(feature_data_path, target_feature + f"_xgb_{n}" + target_feature_suffix)

X = np.load(os.path.join(feature_data_path, target_feature + target_feature_suffix))
feature_importance_use = np.load(f"1048576_seed42_xgb_basic_feature_importance_mean.npy")
selected_indices = np.argsort(feature_importance_use)[-n:][::-1]
X_selected = X[:, selected_indices]
np.save(save_file_name, X_selected)

print(f"saved result to {save_file_name} with shape {X_selected.shape}")

# Prepare geography plot

In [5]:
sample_annotation_df = pd.read_csv(sample_annotation_file, sep="\t")

In [6]:
cls_code_to_name = sample_annotation_df[["Population code", "Population name"]].drop_duplicates().reset_index(drop= True)
# cls_code_to_name

In [7]:
locations = {
    'FIN': (25.0, 60.0),  # Finnish
    'GBR': (-0.1276, 51.5074),  # British,English
    'CHS': (113.7633, 22.18),  # Southern Han Chinese
    'PUR': (-66.1057, 18.4655),  # Puerto Rican
    'CDX': (100.212, 22.0000),  # Dai Chinese
    'CLM': (-74.0817, 4.7110),  # Colombian
    'IBS': (-3.7492, 40.4637),  # Iberian,Spanish
    'KHV': (105.8542, 21.0285),  # Kinh Vietnamese
    'PEL': (-77.0428, -12.0464),  # Peruvian
    'PJL': (74.3587, 31.5204),  # Punjabi
    'ACB': (-61.2225, 10.6918),  # African Caribbean
    'GWD': (-15.3101, 13.4432),  # Gambian Mandinka
    'ESN': (6.6745, 6.7439),  # Esan
    'BEB': (90.4125, 23.8103),  # Bengali
    'MSL': (-11.7383, 7.9647),  # Mende
    'ITU': (78.4867, 17.3850),  # Telugu
    'STU': (80.2707, 13.0827),  # Tamil
    'CEU': (2.3522, 48.8566),  # CEPH
    'YRI': (3.9470, 7.3775),  # Yoruba
    'CHB': (116.4074, 39.9042),  # Han Chinese
    'JPT': (139.6917, 35.6895),  # Japanese
    'LWK': (34.7519, 0.2827),  # Luhya
    'MXL': (-99.1332, 19.4326),  # Mexican Ancestry
    'ASW': (-118.2437, 34.0522),  # African Ancestry SW
    'TSI': (11.2558, 43.7696),  # Toscani
    'GIH': (72.5714, 23.0225)  # Gujarati
}  # Population code -> (longitude, latitude)
color_palette = ["#2A363B", "#355C7D", "#99B898", "#E84A5F", "#FF847C", "#FECEAB"]

In [8]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from matplotlib.patches import Wedge, Circle

def draw_pie_charts_on_map(cities, labs, save_file_name = None, y_log_trans = False):
    """
    주어진 도시 목록에 대해 고해상도 세계지도 위에 파이 차트를 그립니다.

    Parameters:
    cities (list of dicts): 도시 정보를 담고 있는 리스트. 각 항목은 다음 키를 가짐:
        - name (str): 도시 이름
        - lon (float): 경도
        - lat (float): 위도
        - value (float): 파이 차트에서 첫 번째 조각의 비율
        - radius (float): 파이 차트의 크기
    """
    start_angle = 0

    # 평면 지도 생성
    fig, ax = plt.subplots(figsize=(15, 8), subplot_kw={'projection': ccrs.PlateCarree()})
    ax.set_global()

    # High-resolution coastlines and borders
    coastline = cfeature.NaturalEarthFeature('physical', 'coastline', '110m', edgecolor='black', facecolor='none', linewidth=0.5)
    borders = cfeature.NaturalEarthFeature('cultural', 'admin_0_boundary_lines_land', '110m', edgecolor='black', facecolor='none', linewidth=0.2)
    ax.add_feature(coastline)
    ax.add_feature(borders)
    ax.add_feature(cfeature.LAND, color='lightgrey')
    ax.add_feature(cfeature.OCEAN, color='white')

    if y_log_trans:
        radius_normalize = np.mean([np.log10(city["radius"]) for city in cities]) / 4
    else:
        radius_normalize = np.mean([city["radius"] for city in cities]) / 4

    # 각 도시에 대해 파이 차트 그리기
    for city in cities:
        if y_log_trans:
            r = np.log10(city['radius'])
        else:
            r = city['radius']

        if "th2" in city:
            sizes = [city['th1'], city['th2'], 1-city['th1']-city['th2']]  # Proportions for the pie chart
            colors = [color_palette[1], color_palette[2], color_palette[3]]  # Colors for train, validation, and test
        else:
            sizes = [city['th1'], 1 - city['th1']]
            colors = [color_palette[1], color_palette[3]]
        for size, color in zip(sizes, colors):
            angle = 360 * size
            wedge = Wedge(center=(city['lon'], city['lat']), r= r/ radius_normalize,
                          theta1=start_angle, theta2=start_angle + angle,
                          edgecolor='white', linewidth = 0.5, facecolor=color, transform=ccrs.Geodetic())
            ax.add_patch(wedge)
            start_angle += angle
        # ax.text(city['lon'], city['lat'], city['name'], fontsize=8, ha='center', transform=ccrs.Geodetic())

    # Add legend
    theta_legends = []
    theta_legends.append(plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=color_palette[1], markersize=10, label=labs["th1"]))
    if "th2" in city:
        theta_legends.append(plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=color_palette[2], markersize=10, label=labs["th2"]))
    theta_legends.append(plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=color_palette[3], markersize=10, label=labs["th_other"]))
    plt.legend(handles=theta_legends, loc=(0.05,0.05))

    # Add legend for circle sizes
    max_radius = max(city['radius'] for city in cities)
    min_radius = min(city['radius'] for city in cities)
    if y_log_trans:
        legend_sizes = np.logspace(np.log10(min_radius), np.log10(max_radius), num=3)

    else:
        legend_sizes = np.arange(100, max_radius + 50, 50)#np.linspace(min_radius, max_radius, num=3)

    gap_between_edges = 3  # Fixed gap between the edges of the circles
    current_y = -57 + (len(theta_legends) - 3) * 7

    for i, r in enumerate(legend_sizes):
        r_val = np.log10(r) if y_log_trans else r
        #legend location calcuation
        loc_x = -155.5
        loc_y = current_y + r_val / radius_normalize
        current_y += (r_val / radius_normalize) * 2 + gap_between_edges

        

        circ = Circle((loc_x, loc_y), radius=r_val / radius_normalize, edgecolor='black', facecolor='lightgrey', transform=ccrs.Geodetic())
        ax.add_patch(circ)
        plt.text(loc_x + 10, loc_y, f'{round(r)} {labs["radius"]}', fontsize=10, ha='left', va='center', transform=ccrs.Geodetic())
        

            
    if save_file_name is not None:
        plt.savefig(save_file_name, dpi = 300)
    plt.show()


# Fig1. Dataset description

In [9]:
y_train, y_val, y_test = y_original[train_indices], y_original[val_indices], y_original[test_indices]

In [10]:
total_counts = pd.DataFrame(y_original, columns=["class_label"]).value_counts().reset_index(name='Total_Count')
train_counts = pd.DataFrame(y_train, columns=["class_label"]).value_counts().reset_index(name='Train_Count')
val_counts = pd.DataFrame(y_val, columns=["class_label"]).value_counts().reset_index(name='Val_Count')
test_counts = pd.DataFrame(y_test, columns=["class_label"]).value_counts().reset_index(name='Test_Count')

# Merge the counts into a single dataframe
combined_counts = pd.merge(total_counts, train_counts, on='class_label', how='outer')
combined_counts = pd.merge(combined_counts, val_counts, on='class_label', how='outer')
combined_counts = pd.merge(combined_counts, test_counts, on='class_label', how='outer')


In [11]:
combined_counts

,class_label,Total_Count,Train_Count,Val_Count,Test_Count
0,CEU,179,107,36,36
1,YRI,178,107,35,36
2,GWD,178,107,35,36
3,CHS,163,98,32,33
4,IBS,156,94,31,31
5,ESN,149,89,30,30
6,PJL,146,88,29,29
7,PUR,139,83,28,28
8,CLM,132,80,26,26
9,BEB,131,79,26,26


In [ ]:
cities = []
for idx, row in combined_counts.iterrows():
    code = row['class_label']
    if code in locations:
        lon, lat = locations[code]
        city = {
            'name': code,
            'lon': lon,
            'lat': lat,
            'th1': row["Train_Count"] / row["Total_Count"],  # This is an arbitrary placeholder, adjust as necessary
            'th2': row["Val_Count"] / row["Total_Count"],  # This is an arbitrary placeholder, adjust as necessary
            'radius': row["Total_Count"],#  # Normalize by the largest possible value for visualization scaling
        }
        cities.append(city)
    else:
        print(code)


draw_pie_charts_on_map(cities, labs = {"th1": "Train", "th2" : "Validation", "th_other" : "Test", "radius" : "samples"}, save_file_name="results/data_description.pdf")

# Fig4. Number of features required for classifying each class

In [ ]:
df_minimum_n_by_cls = pd.read_csv("results/minimum_n_by_cls.csv")


In [ ]:
standard_class = "FIN"
standard_n = df_minimum_n_by_cls[df_minimum_n_by_cls["class_target"] == standard_class]["select_n"].iloc[0]

feature_importance_standard = np.load(f"8192_seed42_xgb_cls_{standard_class}_basic_feature_importance_mean.npy")
selected_indices_standard = np.argsort(feature_importance_standard)[-standard_n:][::-1]

In [ ]:
selected_indices_standard

In [ ]:
overlab_dict = {}

for i, r in df_minimum_n_by_cls.iterrows():
    current_class = r["class_target"]
    current_n = r["select_n"]
    if current_class == standard_class:
        overlab_dict[current_class] = 1
    feature_importance_current = np.load(f"8192_seed42_xgb_cls_{current_class}_basic_feature_importance_mean.npy")
    selected_indices_current = np.argsort(feature_importance_current)[-current_n:][::-1]
    intersection = np.intersect1d(selected_indices_standard, selected_indices_current)
    overlab_dict[current_class] = len(intersection) / current_n


In [ ]:
# Generate cities data list from the dataframe and location dictionary
cities = []
for idx, row in df_minimum_n_by_cls.iterrows():
    code = row['class_target']
    if code in locations:
        lon, lat = locations[code]
        city = {
            'name': code,
            'lon': lon,
            'lat': lat,
            'th1': 1- overlab_dict[code],  # This is an arbitrary placeholder, adjust as necessary
            'radius': row['select_n']  # Normalize by the largest possible value for visualization scaling
        }
        cities.append(city)
    else:
        print(code)


draw_pie_charts_on_map(cities, y_log_trans = True, labs = {"th1": "other SNPs", "th_other": "SNPs overlap", "radius": "SNPs"}, save_file_name="results/SNP_overlap.pdf")